##### Copyright 2020 The TensorFlow Quantum Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[link text](https://)# Binary classification of quantum states

Initial Tutorial Author : Antonio J. Martinez

Initial Tutorial Contributors : Masoud Mohseni

Initial Tutorial Created : 2020-Feb-14

Initial Tutorial Last updated : 2020-Feb-29

---

Current Experiment Author : Anneliese Brei

Current Experiment Created : 2022-Jan-3

Current Experiment Last updated : 2022-Jan-3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorflow/quantum/blob/research/binary_classifier/binary_classifier.ipynb)

An elementary learning task is [binary classification](https://en.wikipedia.org/wiki/Binary_classification), a supervised task in which the learner is to distinguish which of two classes a given datapoint has been drawn from.  Here, using ideas from the paper [Universal discriminative quantum neural networks](https://arxiv.org/abs/1805.08654) in the one-qubit setting, we train a hybrid quantum-classical neural network to distinguish between quantum data sources.

## Import dependencies

In [ ]:
!pip install --upgrade tensorflow
!pip install qutip

     |████████████████████████████████| 462 kB 6.8 MB/s 
     |████████████████████████████████| 14.8 MB 4.7 MB/s 


In [ ]:
!pip install tensorflow-quantum

     |████████████████████████████████| 10.5 MB 6.9 MB/s 
     |████████████████████████████████| 100 kB 9.3 MB/s 
     |████████████████████████████████| 90 kB 10.5 MB/s 
     |████████████████████████████████| 6.1 MB 27.6 MB/s 
     |████████████████████████████████| 90 kB 8.1 MB/s 
     |████████████████████████████████| 437 kB 45.7 MB/s 
     |████████████████████████████████| 1.6 MB 43.7 MB/s 
     |████████████████████████████████| 93 kB 1.1 MB/s 
     |████████████████████████████████| 93 kB 1.7 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 93 kB 1.5 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 93 kB 891 kB/s 
     |████████████████████████████████| 93 kB 1.2 MB/s 
     |████████████████████████████████| 92 kB 669 kB/s 
     |████████████████████████████████| 93 kB 692 kB/s 
     |████████████████████████████████

In [ ]:
import cirq
import numpy as np
import qutip
import random
import sympy
import tensorflow as tf
import tensorflow_quantum as tfq

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## Quantum dataset
For our quantum dataset, you will generate two blobs on the surface of the Bloch sphere.  The task will be to learn a model to distinguish members of these blobs.  To do this, you first select two axes in the X-Z plane of the block sphere, then select random points uniformly distributed around them:

In [ ]:
def generate_dataset(qubit, theta_a, theta_b, num_samples):
  """Generate a dataset of points on `qubit` near the two given angles; labels
  for the two clusters use a one-hot encoding.
  """
  q_data = []
  bloch = {"a": [[], [], []], "b": [[], [], []]}
  labels = []
  blob_size = abs(theta_a - theta_b) / 5
  for _ in range(num_samples):
    coin = random.random()
    spread_x = np.random.uniform(-blob_size, blob_size)
    spread_y = np.random.uniform(-blob_size, blob_size)
    if coin < 0.5:
      label = [1, 0]
      angle = theta_a + spread_y
      source = "a"
    else:
      label = [0, 1]
      angle = theta_b + spread_y
      source = "b"
    labels.append(label)
    q_data.append(cirq.Circuit(cirq.ry(-angle)(qubit), cirq.rx(-spread_x)(qubit)))
    bloch[source][0].append(np.cos(angle))
    bloch[source][1].append(np.sin(angle)*np.sin(spread_x))
    bloch[source][2].append(np.sin(angle)*np.cos(spread_x))
  return tfq.convert_to_tensor(q_data), np.array(labels), bloch

In [ ]:
def build_model(theta_a, theta_b):

  qubit = cirq.GridQubit(0, 0)

  # Build the quantum model layer 1
  theta = sympy.Symbol('theta')
  q_model = cirq.Circuit(cirq.ry(theta)(qubit))
  q_data_input = tf.keras.Input(
      shape=(), dtype=tf.dtypes.string)
  expectation = tfq.layers.PQC(q_model, cirq.Z(qubit))
  expectation_output = expectation(q_data_input)

  # Attach the classical SoftMax classifier
  classifier = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
  classifier_output = classifier(expectation_output)
  model = tf.keras.Model(inputs=q_data_input, outputs=classifier_output)

  # Standard compilation for classification
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  
  return model, qubit

In [ ]:
# Constants
theta_a = 1
theta_b = 4
num_samples = 500 # Number of datapoints for training and for testing
iterations = 10
num_epochs = 15

Test effect of increasing the number of samples for
1. Training
2. **Testing**

In [ ]:
def train_model():

  # Train with 250 datapoints
  qubit = cirq.GridQubit(0, 0)
  q_data, labels, _ = generate_dataset(qubit, theta_a, theta_b, 250)
  model, _ = build_model(theta_a, theta_b)
  history = model.fit(x=q_data, y=labels, epochs=1, verbose=0)

  return qubit, model


In [ ]:
def run_trial():

  history_p = []

  qubit1, model1 = train_model()
  qubit250, model250 = train_model()
  qubit500, model500 = train_model()

  # Test with 1 datapoint
  test_data, test_labels, test_bloch_p = generate_dataset(qubit1, theta_a, theta_b, 1)
  test_results1 = model1.evaluate(test_data, test_labels, verbose=0)
  history_p.append(test_results1[0])

  # Test with 250 datapoints
  test_data, test_labels, test_bloch_p = generate_dataset(qubit250, theta_a, theta_b, 1)
  test_results250 = model250.evaluate(test_data, test_labels, verbose=0)
  history_p.append(test_results250[0])

  # Test with 500 datapoints
  test_data, test_labels, test_bloch_p = generate_dataset(qubit500, theta_a, theta_b, 1)
  test_results500 = model500.evaluate(test_data, test_labels, verbose=0)
  print(test_results500)
  history_p.append(test_results500[0])

  return history_p

In [ ]:
trial1 = run_trial()
#trial2 = run_trial()
#trial3 = run_trial()
#trial4 = run_trial()
#trial5 = run_trial()
#trial6 = run_trial()
#trial7 = run_trial()
#trial8 = run_trial()
#trial9 = run_trial()
#trial10 = run_trial()

print(trial1)
#print(trial2)
#print(trial3)
#print(trial4)
#print(trial5)
#print(trial6)
#print(trial7)
#print(trial8)
#print(trial9)
#print(trial10)

[0.13466107845306396, 1.0]
[0.16805526614189148, 0.1803816705942154, 0.13466107845306396]


In [ ]:
average = []
for i in range(len(trial1)):
  average.append((trial1[i] + trial2[i] + trial3[i] + trial4[i] + trial5[i]
                  + trial6[i] + trial7[i] + trial8[i] + trial9[i] + trial10[i]) / 10)

print(average)

NameError: ignored

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = ['1', '250', '500']
ax.bar(labels,average)
plt.ylim(.32, .42)
plt.title("Effect of Increasing Testing Data")
plt.xlabel("Size of testing data set")
plt.ylabel("Error in testing classification")
plt.show()

In [ ]:
plt.plot(average, label="training data points")
plt.title("Effect of Increasing Training Data")
plt.xlabel("Size of training data set")
plt.ylabel("Error in testing classification")
plt.show()